In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10, 6)

In [3]:
df = pd.read_csv('nasdaq_new.csv', parse_dates=['Date'], index_col='Date')
# df['Price'] = df['Open Price'].fillna(df['Close'].mean())
cols_to_fill = ['Open Price', 'Daily High', 'Daily Low', 'Trading Volume', 'USD/EUR', 'Crude Oil', 'Gold']
for col in cols_to_fill:
    # df[col].fillna(df[col].median(), inplace=True)
    df[col] = df[col].interpolate(method='linear')

def parse_shorthand(val):
    if isinstance(val, str):
        val = val.strip()
        if val.endswith('B'):
            return float(val[:-1]) * 1e9
        elif val.endswith('M'):
            return float(val[:-1]) * 1e6
        elif val.endswith('K'):
            return float(val[:-1]) * 1e3
        else:
            try:
                return float(val.replace(',', ''))
            except:
                return None
    return val

df['Trading Volume'] = df['Trading Volume'].apply(parse_shorthand)

df.dropna()

C:\Users\bhavi\AppData\Local\Temp\ipykernel_37868\3833666971.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('nasdaq_new.csv', parse_dates=['Date'], index_col='Date')
C:\Users\bhavi\AppData\Local\Temp\ipykernel_37868\3833666971.py:6: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[col] = df[col].interpolate(method='linear')


,Price,Open Price,Daily High,Daily Low,Trading Volume,USD/EUR,Gold,Crude Oil
Date,,,,,,,,
2025-04-16,"16,307.16","16,499.69","16,600.37","16,066.46",1.340000e+09,0.8775,3346.4,62.47
2025-04-15,"16,823.17","16,842.39","16,979.96","16,753.21",1.160000e+09,0.8863,3240.4,61.33
2025-04-14,"16,831.48","17,120.44","17,136.55","16,661.26",1.360000e+09,0.8811,3226.3,61.53
2025-04-11,"16,724.46","16,358.53","16,753.41","16,228.04",1.530000e+09,0.8802,3244.6,61.50
2025-04-10,"16,387.31","16,635.45","16,712.37","15,894.26",1.880000e+09,0.8929,3177.5,60.07
...,...,...,...,...,...,...,...,...
2006-06-21,"2,141.20","2,109.22","2,152.56","2,107.06",1.420000e+09,0.7897,591.0,70.33
2006-06-20,"2,107.06","2,110.92","2,126.72","2,103.77",1.250000e+09,0.7948,580.5,68.94
2006-06-19,"2,110.42","2,136.60","2,138.77","2,104.38",1.250000e+09,0.7950,572.4,68.98


In [4]:
df1 = df.copy()
df1['Daily High'] = df1['Daily High'].str.replace(',', '').astype(float)
df1['Daily Low'] = df1['Daily Low'].str.replace(',', '').astype(float)
df1['Price'] = df1['Price'].str.replace(',', '').astype(float)
df1['Open Price'] = df1['Open Price'].str.replace(',', '').astype(float)
df1['Volatility'] = df1['Daily High'] - df1['Daily Low']
df1.head()

,Price,Open Price,Daily High,Daily Low,Trading Volume,USD/EUR,Gold,Crude Oil,Volatility
Date,,,,,,,,,
2025-04-16,16307.16,16499.69,16600.37,16066.46,1.340000e+09,0.8775,3346.4,62.47,533.91
2025-04-15,16823.17,16842.39,16979.96,16753.21,1.160000e+09,0.8863,3240.4,61.33,226.75
2025-04-14,16831.48,17120.44,17136.55,16661.26,1.360000e+09,0.8811,3226.3,61.53,475.29
2025-04-11,16724.46,16358.53,16753.41,16228.04,1.530000e+09,0.8802,3244.6,61.50,525.37
2025-04-10,16387.31,16635.45,16712.37,15894.26,1.880000e+09,0.8929,3177.5,60.07,818.11


In [5]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

df2 = df1.copy()
df2 = remove_outliers_iqr(df2, 'Volatility')
df2 = remove_outliers_iqr(df2, 'Gold')
df2 = remove_outliers_iqr(df2, 'Crude Oil')
df2 = remove_outliers_iqr(df2, 'USD/EUR')

closePrice = df2['Price']
df2.drop(columns=['Price'], inplace=True)
closePrice

Date
2024-09-09    16884.60
2024-09-04    17084.30
2024-08-30    17713.62
2024-08-16    17631.72
2024-08-15    17594.50
                ...   
2006-06-21     2141.20
2006-06-20     2107.06
2006-06-19     2110.42
2006-06-16     2129.95
2006-06-15     2144.15
Name: Price, Length: 4321, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df2, closePrice, test_size=0.2, random_state=10)

,Open Price,Daily High,Daily Low,Trading Volume,USD/EUR,Gold,Crude Oil,Volatility
Date,,,,,,,,
2024-09-09,16835.67,16923.28,16732.84,1.040000e+09,0.9062,2509.3,67.91,190.44
2024-09-04,17015.71,17232.65,16984.67,9.667000e+08,0.9023,2502.7,68.52,247.98
2024-08-30,17650.49,17720.38,17498.79,1.360000e+09,0.9051,2504.5,72.65,221.59
2024-08-16,17516.40,17674.65,17502.82,9.897500e+08,0.9067,2503.6,75.54,171.83
2024-08-15,17394.54,17602.72,17375.41,1.130000e+09,0.9113,2458.6,76.99,227.31


In [ ]:
from sklearn.ensemble import VotingRegressor

model1 = LinearRegression()
model2 = Lasso()
model3 = DecisionTreeRegressor()

# takes mean of predictions of all models
ensemble = VotingRegressor(
    estimators=[('lr', model1), ('lasso', model2), ('dt', model3)]
)

ensemble.fit(X_train, Y_train)
predictions = ensemble.predict(X_test)

ensemble.score(X_test, Y_test)
# ensemble.predict([[4402.30,	4440.16,	4323.75,	424789909,	0.96,	87.41,	1819.39,	116.41]])

def predict_price(data):
    return ensemble.predict([data])

# predict_price([2644.38,	2682.99,	2569.35,	965696140,	1.06,	90.36,	2143.77,	113.64])


0.9999210070139083

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(ensemble, df2, closePrice, cv=5)
print(scores.mean())

0.8324819797277317


c:\Users\bhavi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.173e+06, tolerance: 1.460e+06
  model = cd_fast.enet_coordinate_descent(


In [11]:
import pickle

with open("stock_price_prediction_model.pkl", "wb") as f:
    pickle.dump(ensemble, f)